## Exercício de Regressão Múltipla – Happiness Dataset

O banco de dados “happiness.xls” contém dados de grau de felicidade de 122 países.
Objetivo: prever o escore de felicidade com base nas variáveis:
- Hemisfério
- HDI
- GDP per capita
- Consumo de cerveja, álcool e vinho per capita

In [4]:
# Importando pacotes
import pandas as pd
import numpy as np
!pip install scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

^C


ModuleNotFoundError: No module named 'sklearn'

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     -------------------------------------- 60.8/60.8 kB 537.5 kB/s eta 0:00:00
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.1 MB 4.5 MB/s eta 0:00:03
    --------------------------------------- 0.3/11.1 MB 3.4 MB/s eta 0:00:04
   - -------------------------------------- 0.5/11.1 MB 4.5 MB/s eta 0:00:03
   --- ------------------------------------ 0.9/11.1 MB 5.1 MB/s eta 0:00:02
   ---- ----------------------------------- 1.2/11.1 MB 6.1 MB/s eta 0:00:02
   ----- ---------------------------------- 1.6/11.1 MB 6.2 MB/s eta 0:00:02
   ------- -------------------------------- 2.0/11.1 MB 6.7 MB/s eta 0:00:02
   -------- ------------------------------- 2.3/11.1 MB 6.5 MB/s eta 0:00:02
   --------- ---

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\Gabi\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\Gabi\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\Gabi\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\Gabi\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Gabi\AppData\Local\Programs\Python\Python312\Lib\http\client.py", line 479, in read
    s = self.fp.read(amt)